In [ ]:
!pip install langchain_community langchain_openai langchain_huggingface langchain_core docx2txt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 449.6/449.6 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
  Attempting uninstall: langchain_core
    Found existing installation: langchain-core 0.3.77
    Uninstalling langchain-core-0.3.77:
      Successfully uninstalled langchain-core-0.3.77
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is

In [40]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
ds = load_dataset("MCINext/synthetic-persian-chatbot-rag-faq-retrieval", "default")
ds

README.md: 0.00B [00:00, ?B/s]

qrels/train.jsonl:   0%|          | 0.00/803k [00:00<?, ?B/s]

qrels/test.jsonl:   0%|          | 0.00/260k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/13044 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4348 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 13044
    })
    test: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 4348
    })
})

In [42]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
corpus = load_dataset("MCINext/synthetic-persian-chatbot-rag-faq-retrieval", "corpus")
corpus

DatasetDict({
    corpus: Dataset({
        features: ['_id', 'title', 'text'],
        num_rows: 8696
    })
})

In [43]:
from datasets import load_dataset

# Login using e.g. `huggingface-cli login` to access this dataset
queries = load_dataset("MCINext/synthetic-persian-chatbot-rag-faq-retrieval", "queries")
queries

queries.jsonl:   0%|          | 0.00/13.5M [00:00<?, ?B/s]

Generating queries split:   0%|          | 0/4348 [00:00<?, ? examples/s]

DatasetDict({
    queries: Dataset({
        features: ['_id', 'text'],
        num_rows: 4348
    })
})

In [45]:
documents = []
for doc in corpus['corpus']:
    # Combine title and text for better context
    full_text = f"{doc['title']}\n{doc['text']}"
    documents.append({
        'id': doc['_id'],
        'content': full_text,
        'metadata': {'title': doc['title']}
    })

print(f"Total documents: {len(documents)}")
print(f"First document preview:")
print(f"ID: {documents[0]['id']}")
print(f"Content: {documents[0]['content']}...")

Total documents: 8696
First document preview:
ID: 0
Content: 
سوال : آیا می‌تونید بگید که چقدر مصرف روزانه من باید باشه؟
پاسخ : معمولاً مصرف متوسط بستگی به تعداد وسایل و نوع استفاده داره، ولی شماره دقیقی نمی‌تونیم بگیم....


In [48]:
from langchain.schema import Document

# Convert our documents to LangChain Document format
langchain_documents = []

for doc in documents:
    langchain_doc = Document(
        page_content=doc['content'],
        metadata={
            'id': doc['id'],
            'title': doc['metadata']['title']
        }
    )
    langchain_documents.append(langchain_doc)

print(f"Created {len(langchain_documents)} LangChain documents")
print(f"First document type: {type(langchain_documents[0])}")

Created 8696 LangChain documents
First document type: <class 'langchain_core.documents.base.Document'>


In [47]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
    model_kwargs={'device': 'cpu'}
)

In [49]:
from langchain_community.vectorstores import FAISS

# Create FAISS vector store from documents
print("Creating FAISS vector store... (this may take a few minutes)")

vector_store = FAISS.from_documents(
    documents=langchain_documents,
    embedding=embedding_model
)

print("FAISS vector store created successfully!")
print(f"Vector store type: {type(vector_store)}")

# Save the vector store for later use
vector_store.save_local("faiss_persian_vectorstore")
print("Vector store saved to 'faiss_persian_vectorstore' folder")

Creating FAISS vector store... (this may take a few minutes)
FAISS vector store created successfully!
Vector store type: <class 'langchain_community.vectorstores.faiss.FAISS'>
Vector store saved to 'faiss_persian_vectorstore' folder


In [50]:
retriever = vector_store.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 3}
)

In [51]:
from langchain_openai import ChatOpenAI
import os


llm = ChatOpenAI(
    model="gpt-4o",
    api_key = os.getenv('OPENAI_API_KEY'),
    base_url = os.getenv('OPENAI_BASE_URL'),
    temperature=0.1
)

print("OpenAI LLM initialized!")

OpenAI LLM initialized!


In [52]:
from langchain.prompts import PromptTemplate

# Create a Persian RAG prompt template
rag_prompt_template = """
شما یک دستیار هوشمند هستید که باید بر اساس اطلاعات ارائه شده به سوالات پاسخ دهید.

متن مرجع:
{context}

سوال:
{question}

لطفاً بر اساس متن مرجع ارائه شده، پاسخ دقیق و مفصلی به سوال بدهید. اگر پاسخ در متن مرجع موجود نیست، بگویید که اطلاعات کافی در دسترس نیست.

پاسخ:
"""

rag_prompt = PromptTemplate(
    template=rag_prompt_template,
    input_variables=["context", "question"]
)

print("RAG prompt template created!")

RAG prompt template created!


In [53]:
from langchain.chains import RetrievalQA

rag_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    chain_type_kwargs={"prompt": rag_prompt},
    return_source_documents=True
)


In [60]:
queries['queries'][1000]['text']

'کاربر : چطوری می\u200cتونم این گوشی جدیدم رو روشن کنم؟\nچت بات : سلام! خیلی آسونه! فقط باید دکمه\u200cی پاور رو فشار بدی.\nکاربر : کدوم دکمه؟ می\u200cتونی عکسش رو نشونم بدی؟\nچت بات : متاسفانه نمی\u200cتونم عکس بفرستم، اما معمولاً بالای گوشی یه دکمه هست که باید نگهش داری.\nکاربر : وای! این همه دکمه داره! چجوری می\u200cدونم کدومش دکمه\u200cی پاوره؟\nچت بات : دکمه\u200cی پاور معمولاً یه علامت دایره داره که وسطش یه خط میاد.\nکاربر : امتحان می\u200cکنم! اگه روشن نشه چی میشه؟\nچت بات : اگر روشن نشه، ممکنه باتری\u200cش خالی باشه. می\u200cتونی یه بار شارژش کنی.\nکاربر : چقدر شارژش کنم؟\nچت بات : حدودا 30 دقیقه کافیه!\nکاربر : این گوشی چیکار می\u200cکنه؟\nپیام جدید کاربر : می\u200cتونم باهاش بازی کنم؟'

In [74]:
test_query = queries['queries'][1754]['text']
print(f"Question: {test_query}")
print("-" * 50)

result = rag_chain({"query": test_query})

print("RAG Answer:")
print(result['result'])
print("\n" + "="*50)
print("Source Documents Used:")
for i, doc in enumerate(result['source_documents']):
    print(f"\nSource {i+1}:")
    print(f"Content: {doc.page_content[:200]}...")
    print(f"Metadata: {doc.metadata}")

Question: 
پیام جدید کاربر : چرا هیچ کسی به سوالات من جواب نمی‌ده؟ این کالا اصلاً کار نمی‌کنه!
--------------------------------------------------
RAG Answer:
ما تمام تلاش خود را می‌کنیم که به تمام کاربران پاسخ دهیم، اما ممکن است به دلیل ترافیک بالای تماس‌ها پاسخ‌دهی کند باشد. اگر سوال شما درباره عملکرد کالا است و به نظر می‌رسد که کالا کار نمی‌کند، پیشنهاد می‌کنیم با پشتیبانی فنی یا خدمات پس از فروش تماس بگیرید تا مشکل شما به صورت دقیق‌تر بررسی شود. اگر سوال دیگری دارید، لطفاً با جزئیات بیشتری مطرح کنید.

Source Documents Used:

Source 1:
Content: 
سوال : چرا هیچ کس به سوالات من جواب نمی‌دهد؟
پاسخ : ما تمام تلاش خود را می‌کنیم که به تمام کاربران پاسخ دهیم، اما ممکن است به دلیل ترافیک بالای تماس‌ها پاسخ‌دهی کند باشد....
Metadata: {'id': 2225, 'title': ''}

Source 2:
Content: 
سوال : چرا چت بات به سوالات من جواب نمی‌دهد؟
پاسخ : چت بات به همه سوالات پاسخ می‌دهد، اما ممکن است محتوای سوال شما با اطلاعات موجود در پایگاه داده ما مطابقت نداشته باشد....
Metadata: {'id': 507, 'title': ''}

Source